In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from glob import glob


2024-08-07 01:32:39.835469: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


2024-08-07 01:33:24.036688: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


58889256/58889256 [==============================] - 13s 0us/step


In [22]:
# Add custom layers on top of VGG16 base
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Assuming you have 10 classes
])


In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [31]:
import tensorflow as tf

# Image augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)  # Only rescale test/validation data

# Load images
training_set = train_datagen.flow_from_directory(
    'dataset/train',        # Directory with training images
    target_size=(64, 64),          # Resize all images to 64x64
    batch_size=32,                 # Number of images to be yielded from the generator per batch
    class_mode='binary',            # Type of label arrays to be returned ('binary' for binary classification)
    subset='training'
)

test_set = test_datagen.flow_from_directory(
    'dataset/test',            # Directory with test/validation images
    target_size=(64, 64),          # Resize all images to 64x64
    batch_size=32,                 # Number of images to be yielded from the generator per batch
    class_mode='binary',            # Type of label arrays to be returned ('binary' for binary classification)
    subset='validation'
)

Found 2742 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [32]:
history = model.fit(
    training_set,
    steps_per_epoch=training_set.samples, # train_generator.batch_size,
    validation_data=test_set,
    validation_steps=test_set.samples, # validation_generator.batch_size,
    epochs=1
)


2024-08-07 02:21:48.383003: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


  57/2742 [..............................] - ETA: 3:05:23 - loss: 0.6540 - accuracy: 0.6488

KeyboardInterrupt: 

In [30]:
# Evaluate the model
val_loss, val_acc = model.evaluate(test_set)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")

# Predict the classes
y_pred = model.predict(test_set)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels
y_true = test_set.classes

# Generate classification report and confusion matrix
print(classification_report(y_true, y_pred_classes, target_names=test_set.class_indices.keys()))
print(confusion_matrix(y_true, y_pred_classes))


ValueError: Asked to retrieve element 0, but the Sequence has length 0